# 📚 Cifrado ISO 27017 con AWS KMS y S3

## Introducción sobre KMS y ISO 27017
### KMS (Key Management Service)

- 🔒 Servicio de AWS para crear y gestionar claves criptográficas.
- 🔐 Protege datos cifrados en servicios como S3.

## ISO 27017
- 🌐 Estándar de seguridad en la nube.
- 📜 Proporciona directrices para proteger datos en la nube.
- ✅ KMS ayuda a cumplir con ISO 27017 cifrando y protegiendo datos sensibles.

## Descripción demo
- 📦 Desarrollar la clase KMSS3Manager para manejar claves KMS y buckets S3.
- 🔑 Métodos incluidos para:
    - Crear claves KMS.
    - Crear buckets S3.
    - Configurar cifrado del bucket con KMS.
    - Subir y descargar archivos cifrados.

## 1. Importar bibliotecas necesarias
#### En esta sección importamos las bibliotecas necesarias para interactuar con AWS KMS y S3.


In [2]:
import boto3
import json
import uuid
from botocore.exceptions import ClientError

## 2. Definir la clase KMSS3Manager
#### Definimos una clase que manejará la creación y configuración de claves KMS y buckets S3.


In [3]:
class KMSS3Manager:
    def __init__(self, region_name='us-east-1'):
        self.kms_client = boto3.client('kms', region_name=region_name)
        self.s3_client = boto3.client('s3', region_name=region_name)
        self.key_id = None
        self.bucket_name = None

## 2.1 Crear clave KMS
#### Este método crea una nueva clave KMS que se usará para cifrar datos en S3.

In [15]:
def create_kms_key(self, description="🔐 Clave para cifrado S3 alineado con ISO 27017"):
    try:
        response = self.kms_client.create_key(
            Description=description,
            KeyUsage='ENCRYPT_DECRYPT',
            Origin='AWS_KMS'
        )
        self.key_id = response['KeyMetadata']['KeyId']
        print(f"🔑 Nueva clave KMS creada. ID: {self.key_id}")
        return self.key_id
    except ClientError as e:
        print(f"❌ Error al crear la clave KMS: {e}")
        return None

# Asignar el método a la clase KMSS3Manager
setattr(KMSS3Manager, "create_kms_key", create_kms_key)


## 2.2 Crear bucket S3
### Este método crea un nuevo bucket S3 con un nombre único, utilizando un prefijo y un UUID.

In [16]:
def create_s3_bucket(self, bucket_name_prefix):
    bucket_name = f"{bucket_name_prefix}-{uuid.uuid4()}"
    try:
        self.s3_client.create_bucket(Bucket=bucket_name)
        self.bucket_name = bucket_name
        print(f"🪣 Bucket S3 creado: {bucket_name}")
        return True
    except ClientError as e:
        print(f"❌ Error al crear el bucket S3: {e}")
        return False

# Asignar el método a la clase KMSS3Manager
setattr(KMSS3Manager, "create_s3_bucket", create_s3_bucket)


## 2.3 Configurar cifrado del bucket S3
#### Este método configura el cifrado del bucket S3 usando la clave KMS creada anteriormente.
   

In [17]:
def configure_s3_encryption(self):
    if not self.key_id or not self.bucket_name:
        print("⚠️ Error: Se requiere una clave KMS y un bucket S3.")
        return False
    try:
        self.s3_client.put_bucket_encryption(
            Bucket=self.bucket_name,
            ServerSideEncryptionConfiguration={
                'Rules': [{
                    'ApplyServerSideEncryptionByDefault': {
                        'SSEAlgorithm': 'aws:kms',
                        'KMSMasterKeyID': self.key_id
                    }
                }]
            }
        )
        print(f"🔒 Cifrado configurado para el bucket {self.bucket_name}")
        return True
    except ClientError as e:
        print(f"❌ Error al configurar el cifrado del bucket: {e}")
        return False

# Asignar el método a la clase KMSS3Manager
setattr(KMSS3Manager, "configure_s3_encryption", configure_s3_encryption)


## 2.4 Subir archivo al bucket S3
#### Este método permite subir archivos al bucket S3, asegurando que estén cifrados.

In [18]:
def upload_file(self, file_name, object_name=None):
    if object_name is None:
        object_name = file_name
    try:
        self.s3_client.upload_file(file_name, self.bucket_name, object_name)
        print(f"📤 Archivo {file_name} subido como {object_name}")
        return True
    except ClientError as e:
        print(f"❌ Error al subir el archivo: {e}")
        return False

# Asignar el método a la clase KMSS3Manager
setattr(KMSS3Manager, "upload_file", upload_file)


## 2.5 Descargar archivo del bucket S3
#### Este método descarga archivos desde el bucket S3, verificando que estén cifrados.

In [19]:
def download_file(self, object_name, file_name):
    try:
        self.s3_client.download_file(self.bucket_name, object_name, file_name)
        print(f"📥 Archivo {object_name} descargado como {file_name}")
        return True
    except ClientError as e:
        print(f"❌ Error al descargar el archivo: {e}")
        return False

# Asignar el método a la clase KMSS3Manager
setattr(KMSS3Manager, "download_file", download_file)


## 2.6 Verificar cifrado del bucket S3
#### Este método comprueba que el bucket S3 está configurado correctamente para usar cifrado KMS.

In [20]:
def verify_bucket_encryption(self):
    try:
        response = self.s3_client.get_bucket_encryption(Bucket=self.bucket_name)
        rules = response['ServerSideEncryptionConfiguration']['Rules']
        for rule in rules:
            if rule['ApplyServerSideEncryptionByDefault']['SSEAlgorithm'] == 'aws:kms' and rule['ApplyServerSideEncryptionByDefault']['KMSMasterKeyID'] == self.key_id:
                print("✅ El bucket está correctamente configurado para usar KMS.")
                return True
        print("❌ El bucket no está configurado correctamente para usar KMS.")
        return False
    except ClientError as e:
        print(f"❌ Error al verificar el cifrado del bucket: {e}")
        return False

# Asignar el método a la clase KMSS3Manager
setattr(KMSS3Manager, "verify_bucket_encryption", verify_bucket_encryption)


## 2.7 Verificar cifrado del objeto en el bucket S3
### Este método verifica que los objetos dentro del bucket S3 estén cifrados usando la clave KMS.

In [21]:
def verify_object_encryption(self, object_name):
    try:
        response = self.s3_client.head_object(Bucket=self.bucket_name, Key=object_name)
        if 'ServerSideEncryption' in response and response['ServerSideEncryption'] == 'aws:kms':
            print(f"✅ El objeto {object_name} está cifrado con KMS.")
            return True
        print(f"❌ El objeto {object_name} no está cifrado con KMS.")
        return False
    except ClientError as e:
        print(f"❌ Error al verificar el cifrado del objeto: {e}")
        return False

# Asignar el método a la clase KMSS3Manager
setattr(KMSS3Manager, "verify_object_encryption", verify_object_encryption)


# 🌟 Ejemplo de uso

## 3. Inicializar el gestor KMSS3Manager
#### Creamos una instancia de la clase KMSS3Manager para gestionar nuestras operaciones

In [11]:
manager = KMSS3Manager()

## 4. Crear clave KMS
#### Llamamos al método para crear una clave KMS.

In [14]:
key_id = manager.create_kms_key()

🔑 Nueva clave KMS creada. ID: 8b2b9b35-bd71-4e93-83fb-f92ace68a32d


## 5. Crear bucket S3 con un prefijo y UUID único
#### Usamos un prefijo para el nombre del bucket y creamos un bucket único.

In [22]:
bucket_prefix = "bucket-sfe-test"
manager.create_s3_bucket(bucket_prefix)

🪣 Bucket S3 creado: bucket-sfe-test-d467cb8f-25a5-4c03-8ddc-2a8a78313f86


True

## 6. Configurar cifrado del bucket
#### Configuramos el cifrado del bucket utilizando la clave KMS creada

In [23]:
manager.configure_s3_encryption()

🔒 Cifrado configurado para el bucket bucket-sfe-test-d467cb8f-25a5-4c03-8ddc-2a8a78313f86


True

## 7. Verificar la configuración de cifrado del bucket
#### Verificamos que el bucket está configurado correctamente para usar cifrado KMS.

In [24]:
manager.verify_bucket_encryption()

✅ El bucket está correctamente configurado para usar KMS.


True

## 8. Subir un archivo al bucket S3
#### Subimos un archivo al bucket S3. (Asumiendo que existe un archivo 'documento_secreto.txt')

In [25]:
manager.upload_file('assets/documento_secreto.txt')

📤 Archivo assets/documento_secreto.txt subido como assets/documento_secreto.txt


True

## 9. Verificar el cifrado del objeto
#### Verificamos que el archivo subido está cifrado con la clave KMS.

In [26]:
manager.verify_object_encryption('assets/documento_secreto.txt')

✅ El objeto assets/documento_secreto.txt está cifrado con KMS.


True

## 10. Descargar el archivo del bucket S3
#### Descargamos el archivo desde el bucket S3.

In [27]:
manager.download_file('assets/documento_secreto.txt', 'documento_descargado.txt')

📥 Archivo assets/documento_secreto.txt descargado como documento_descargado.txt


True

In [28]:
print("🚀 Flujo completado.")

🚀 Flujo completado.
